# Case Study on SQL


In [2]:
import sqlite3
import pandas as pd

In [10]:
def load_imdb_database(db_path):
  try:
    conn = sqlite3.connect(db_path)
    print("IMDB database loaded successfully.")
    return conn
  except sqlite3.Error as e:
    print(f"Error loading IMDB database: {e}")
    return None

In [3]:
db_path = "imdb.db"
conn = load_imdb_database(db_path)

if conn:
  print("Connection successful")
  cursor = conn.cursor()

IMDB database loaded successfully.
Connection successful


## Following is the schema of the IMDB database. It lists all tables and their schemas.



In [4]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Database Schema:")
for table in tables:
    table_name = table[0]
    print(f"\nTable: {table_name}")

    cursor.execute(f"PRAGMA table_info({table_name});")
    schema = cursor.fetchall()

    print("Columns:")
    for column in schema:
        cid, name, dtype, notnull, dflt_value, pk = column
        print(f"  - {name} ({dtype}), NOT NULL: {bool(notnull)}, Default: {dflt_value}, Primary Key: {bool(pk)}")

Database Schema:

Table: Movie
Columns:
  - index (INTEGER), NOT NULL: False, Default: None, Primary Key: False
  - MID (TEXT), NOT NULL: False, Default: None, Primary Key: False
  - title (TEXT), NOT NULL: False, Default: None, Primary Key: False
  - year (TEXT), NOT NULL: False, Default: None, Primary Key: False
  - rating (REAL), NOT NULL: False, Default: None, Primary Key: False
  - num_votes (INTEGER), NOT NULL: False, Default: None, Primary Key: False

Table: Genre
Columns:
  - index (INTEGER), NOT NULL: False, Default: None, Primary Key: False
  - Name (TEXT), NOT NULL: False, Default: None, Primary Key: False
  - GID (INTEGER), NOT NULL: False, Default: None, Primary Key: False

Table: Language
Columns:
  - index (INTEGER), NOT NULL: False, Default: None, Primary Key: False
  - Name (TEXT), NOT NULL: False, Default: None, Primary Key: False
  - LAID (INTEGER), NOT NULL: False, Default: None, Primary Key: False

Table: Country
Columns:
  - index (INTEGER), NOT NULL: False, Defau

# Write SQL queries for the following questions

## 1. Write query to list first 5 rows of Person table





In [7]:
q1 = """SELECT * FROM Person LIMIT 5;"""

In [8]:
def q1_grader(q1):
  result = pd.read_sql_query(q1, conn)
  return result.shape == (5, 4)
print(q1_grader(q1))

True


## 2. Write query to select title, year and rating from Movie table

In [13]:
q2 = """SELECT title, year, rating FROM Movie ORDER BY title;"""

In [14]:
def q2_grader(q2):
  result = pd.read_sql_query(q2, conn)
  return result.shape == (3473, 3) and result['title'][0]=='Mowgli'
print(q2_grader(q2))

False


## 3. Write query to get title of first movie in movie table sorted by year in ascending order


In [15]:
q3 = """SELECT title FROM Movie ORDER BY year ASC LIMIT 1;"""

In [16]:
def q3_grader(q3):
  result = pd.read_sql_query(q3, conn)
  return result['title']=='Alam Ara'
print(q3_grader(q3))

0    True
Name: title, dtype: bool


## 4. Write query to get the very first year in which Devdas movie was released

In [17]:
q4 = """SELECT MIN(year) AS year FROM Movie WHERE title = 'Devdas';"""

In [18]:
def q4_grader(q4):
  result = pd.read_sql_query(q4, conn)
  return result['year']=='1936'
print(q4_grader(q4))

0    True
Name: year, dtype: bool


## 5. Write query to get the number of movies released in 2018

In [19]:
q5 = """SELECT COUNT(*) FROM Movie WHERE year = 2018;"""


In [20]:
def q5_grader(q5):
  result = pd.read_sql_query(q5, conn)
  return result.iloc[0, 0] == 93
print(q5_grader(q5))

True


## 6. Write query to get the title of the movie with most number of votes in 2012

In [23]:
q6 = """SELECT title FROM Movie WHERE year = 2012 ORDER BY votes DESC LIMIT 1;"""

In [24]:
def q6_grader(q6):
  result = pd.read_sql_query(q6, conn)
  return result["title"]=="The Avengers"
print(q6_grader(q6))

DatabaseError: Execution failed on sql 'SELECT title FROM Movie WHERE year = 2012 ORDER BY votes DESC LIMIT 1;': no such column: votes

## 7. Write SQL query to find all the unique movie titles released in 2018

In [25]:
q7 = """SELECT DISTINCT title FROM Movie WHERE year = 2018;"""

In [26]:
def q7_grader(q7):
  result = pd.read_sql_query(q7, conn)
  return result.shape==(93, 1)
print(q7_grader(q7))

True


## 8. Write SQL query to get total number of movies released between 2017 (inclusive) and 2018 (inclusive)

In [27]:
q8 = """SELECT COUNT(*) FROM Movie WHERE year BETWEEN 2017 AND 2018;"""

In [28]:
def q8_grader(q8):
  result = pd.read_sql_query(q8, conn)
  return result.iloc[0, 0] == 211
print(q8_grader(q8))

True


## 9. Write SQL query to find the year in which maximum number of movies released

In [29]:
q9 = """SELECT year FROM Movie GROUP BY year ORDER BY COUNT(*) DESC LIMIT 1;"""

In [30]:
def q9_grader(q9):
  result = pd.read_sql_query(q9, conn)
  return result["year"][0]=="2005"
print(q9_grader(q9))

True


## 10. Write SQL query to find the title of the movie with rating>9.5 and number of votes > 90

In [31]:
q10 = """SELECT title FROM Movie WHERE rating > 9.5 AND votes > 90;"""

In [34]:
def q10_grader(q10):
    result = pd.read_sql_query(q10, conn)
    return result["title"][0] == "Man on Mission Fauladi"

print(q10_grader(q10))

DatabaseError: Execution failed on sql 'SELECT title FROM Movie WHERE rating > 9.5 AND votes > 90;': no such column: votes

## 11. Write SQL query to find the number of movies which has the word 'Dilwale' in their title

In [35]:
q11 = """SELECT COUNT(*) FROM Movie WHERE title LIKE '%Dilwale%';"""


In [36]:
def q11_grader(q11):
  result = pd.read_sql_query(q11, conn)
  return result.iloc[0, 0] == 4
print(q11_grader(q11))

True


## 12. Write nested SQL query to find the CID of country which produced most number of movies

In [37]:
q12 = """
SELECT CID
FROM Movie
GROUP BY CID
HAVING COUNT(*) = (
  SELECT MAX(movie_count)
  FROM (
    SELECT COUNT(*) AS movie_count
    FROM Movie
    GROUP BY CID
  )
);
"""

In [38]:
def q12_grader(q12):
  result = pd.read_sql_query(q12, conn)
  return result.iloc[0]==2.0
print(q12_grader(q12))

DatabaseError: Execution failed on sql '
SELECT CID 
FROM Movie 
GROUP BY CID 
HAVING COUNT(*) = (
  SELECT MAX(movie_count) 
  FROM (
    SELECT COUNT(*) AS movie_count 
    FROM Movie 
    GROUP BY CID
  )
);
': no such column: CID

## 13. Write nested SQL query to the country which produced most number of movies (use both Courty table and M_Country table)

In [39]:
q13 = """
SELECT c.name
FROM Country c
WHERE c.CID = (
  SELECT CID
  FROM M_Country
  GROUP BY CID
  ORDER BY COUNT(*) DESC
  LIMIT 1
);
"""


In [40]:
def q13_grader(q13):
  result = pd.read_sql_query(q13, conn)
  return result.iloc[0, 0] == "India"
print(q13_grader(q13))

True


## 14. Write SQL query to get the year and number of movies per year having number of movies per year is greater than 100

In [41]:
q14 = """
SELECT year, COUNT(*) AS num_movies
FROM Movie
GROUP BY year
HAVING COUNT(*) > 100;
"""


In [42]:
def q14_grader(q14):
  result = pd.read_sql_query(q14, conn)
  return result.shape==(13,2)
print(q14_grader(q14))

True


## 15. Write SQL query to get the Name and Language ID (LAID) corresponding to Malayalam language

In [43]:
q15 = """SELECT Name, LAID FROM Language WHERE Name = 'Malayalam';"""

In [44]:
def q15_grader(q15):
  result = pd.read_sql_query(q15, conn)
  print(result)
  return result[["Name", "LAID"]].values.tolist() == [['Malayalam', 19]]
print(q15_grader(q15))

        Name  LAID
0  Malayalam    19
True


## 16. Write SQL query to do inner join with movie table and M_Language table with MID colums

In [45]:
q16 = """SELECT * FROM Movie INNER JOIN M_Language ON Movie.MID = M_Language.MID;"""


In [46]:
def q16_grader(q16):
  result = pd.read_sql_query(q16, conn)
  return result
print(q16_grader(q16))

      index        MID          title  year  rating  num_votes  index  \
0         0  tt2388771         Mowgli  2018     6.6      21967      0   
1         1  tt5164214  Ocean's Eight  2018     6.2     110861      1   
2         2  tt1365519    Tomb Raider  2018     6.4     142585      2   
3         3  tt0848228   The Avengers  2012     8.1    1137529      3   
4         4  tt8239946        Tumbbad  2018     8.5       7483      4   
...     ...        ...            ...   ...     ...        ...    ...   
3468   3470  tt0090611    Allah-Rakha  1986     6.2         96   3470   
3469   3471  tt0106270          Anari  1993     4.7        301   3471   
3470   3472  tt0852989  Come December  2006     5.7         57   3472   
3471   3473  tt0375882     Kala Jigar  1939     3.3        174   3473   
3472   3474  tt0375890         Kanoon  1994     3.2        103   3474   

            MID  LAID    ID  
0     tt2388771     0     0  
1     tt5164214     0     1  
2     tt1365519     0     2  
3  

## 17. Write SQL query to list title, year and rating of malayalam movies in the database by doing an inner join with movie table and M_Language table with MID column, also assuming language ID of malayalam movies as 19

In [47]:
q17 = """
SELECT m.title, m.year, m.rating
FROM Movie m
INNER JOIN M_Language ml ON m.MID = ml.MID
WHERE ml.LAID = 19;
"""


In [48]:
def q17_grader(q17):
  result = pd.read_sql_query(q17, conn)
  return result.shape==(16,3)
print(q17_grader(q17))

True


In [49]:
![ -f academic.db ] && rm academic.db

conn = sqlite3.connect("academic.db")
cursor = conn.cursor()

## 18. Write SQL query to Create a table named students with two columns id (integer type) and name (varchar type)

In [50]:
q18a = """CREATE TABLE students (id INTEGER, name VARCHAR(255));"""


In [51]:
def q18_grader_a(q18):
  try:
    cursor.execute(q18)
    return True
  except:
    pass

def q18_grader_b(q19):
  result = pd.read_sql_query(q19, conn)
  return result.columns.tolist() == ['id', 'name']

q18b = """SELECT * FROM students;"""

print(q18_grader_a(q18a) and q18_grader_b(q18b))

True


## 19. Write SQL query to insert the values (1, 'Alice') into students table

In [52]:
q19_a = """INSERT INTO students (id, name) VALUES (1, 'Alice');"""


In [53]:
def q19_grader_a(q19_a):
  try:
    cursor.execute(q19_a)
  except:
    pass
  return True

def q19_grader_b(q19_b):
  result = pd.read_sql_query(q19_b, conn)
  return result.values.tolist() == [[1, 'Alice']]

q19_b = "SELECT * FROM students"
print(q19_grader_a(q19_a) and q19_grader_b(q19_b))

True


## 20. Write SQL Query to add the following more information to students table

| ID | Name    |
|----|---------|
| 2  | Bob     |
| 3  | Charlie |

In [54]:
q20 = """INSERT INTO students (id, name) VALUES (2, 'Bob'), (3, 'Charlie');"""


In [55]:
def q20_grader(q20):
  try:
    cursor.execute(q20)
  except:
    pass
  return True

def q20_grader_b(q20_b):
  result = pd.read_sql_query(q20_b, conn)
  return result.values.tolist() == [[1, 'Alice'], [2, 'Bob'], [3, 'Charlie']]

print(q20_grader(q20) and q20_grader_b(q19_b))

True


In [56]:
conn.close()